In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd '/content/drive/MyDrive/NLP'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/NLP


In [ ]:
#!pip install sentence-transformers

In [ ]:
import torch
import pandas as pd
import numpy as np
#from sentence_transformers import SentenceTransformer
from sklearn.cluster import KMeans
from sklearn.metrics import v_measure_score
from tqdm import tqdm
import torch.nn.functional as F

In [ ]:
data = pd.read_csv("bibledata.csv")

In [ ]:
model = SentenceTransformer('all-MiniLM-L6-v2')

In [ ]:
emb = model.encode(data.text)
torch.save(emb, "bible_embeddings.pt")

In [ ]:
embeddings = torch.load("bible_embeddings.pt")
similarity_matrix = embeddings.matmul(embeddings.T)

<ipython-input-4-8fe33c705e54>:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  embeddings = torch.tensor(torch.load("bible_embeddings.pt"))


In [ ]:
mean_context = torch.zeros(len(embeddings), 384)
top = 7
for i in tqdm(range(len(embeddings))):
  mean_context[i] = torch.mean(embeddings[similarity_matrix[i].sort()[1][-top:]], 0)
torch.save(mean_context, "bible_similar_mean.pt")

100%|██████████| 10731/10731 [00:12<00:00, 825.83it/s] 


In [ ]:
mean_context = torch.load("bible_similar_mean.pt")
mean_context = F.normalize(mean_context, dim = 1)

In [ ]:
styles = embeddings - mean_context
kmeans_by_style = KMeans(n_clusters = 7)
kmeans_by_style.fit(styles1)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


KMeans(n_clusters=7)

In [ ]:
data["labels"] = kmeans_by_style.labels_
print(v_measure_score(data.id, data.labels))

0.31677207848201633


In [ ]:
#kmeans_by_verse = KMeans(n_clusters = 1533)
#kmeans_by_verse.fit(embeddings)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


KMeans(n_clusters=1533)

In [ ]:
#cluster_centres = kmeans_by_verse.cluster_centers_[kmeans_by_verse.labels_]
#torch.save(cluster_centres, "bible_cluster_centres.pt")

In [ ]:
cluster_centres = torch.tensor(torch.load("bible_cluster_centres.pt"))

In [ ]:
styles2 = embeddings - cluster_centres

In [ ]:
kmeans_by_style2 = KMeans(n_clusters = 7)
kmeans_by_style2.fit(styles2)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


KMeans(n_clusters=7)

In [ ]:
data["labels2"] = kmeans_by_style2.labels_
print(v_measure_score(data.id, data.labels2))

0.32203628868427797
